<a href="https://colab.research.google.com/github/KarineAyrs/science_work/blob/main/training/timm_image_models_resnet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install timm

In [3]:
import timm

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [5]:
model_name = 'resnet34'

In [6]:
timm_model=timm.create_model(model_name=model_name)

timm_model.eval()

x= torch.randn(1, 3, 28, 28)
print(timm_model(x).shape)

torch.Size([1, 1000])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [ ]:
if torch.cuda.is_available():
    timm_model.cuda()

In [ ]:
in_channel=3
num_classes=10
learning_rate=0.001
batch_size=2
num_epochs =1

timm_model.train()

In [10]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader=DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader=DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(timm_model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
  for batch, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data=data.to(device=device)
    targets=targets.to(device=device)
    
    
    data = data.repeat(1,3,1,1)
    
    # forward
    scores = timm_model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()



In [13]:

def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on training data')
  else:
    print('Checking accuracy o test data')
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)

      x=x.repeat(1,3,1,1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct+=(predictions==y).sum()
      num_samples+=predictions.size(0)
    
    print(f'Got {num_correct}/{num_samples} with accuracy {(float(num_correct)/float(num_samples))*100}')
  
  model.train()
 
  

In [14]:
check_accuracy(train_loader, timm_model)
check_accuracy(test_loader, timm_model)

Checking accuracy on training data
Got 18227/60000 with accuracy 30.378333333333334
Checking accuracy o test data
Got 3059/10000 with accuracy 30.59
